# Train Keras model and make performance plots

In [1]:
#Data Processing
import numpy as np
from numpy import loadtxt, expand_dims
import h5py
import sys

#ML model
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense,Flatten, Dropout, Activation, concatenate, BatchNormalization, GRU, Add, Conv1D, Conv2D, Concatenate
from tensorflow.keras.models import Model, Sequential, load_model


from sklearn.metrics import roc_curve
from sklearn.metrics import auc

#Plot settings
%matplotlib inline
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.ROOT)

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'medium',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'medium',
         'ytick.labelsize':'medium'}
pylab.rcParams.update(params)

#line thickness
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 5

#Just to efficiently save plots
plot_dir = "../plot/"
print("Saving plot to: ", plot_dir)
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y")
save_path = plot_dir + dt_string

Saving plot to:  ../plot/


## 1. Train model

In [2]:
#Load in the datasets for training and compiling the sample weights
with h5py.File('../data/trainingDataver3.h5', 'r') as hf:
    dataset = hf["Training Data"][:]
with h5py.File('../data/sampleDataver3.h5', 'r') as hf:
    sampleData = hf["Sample Data"][:]
    

#Separate datasets into inputs and outputs, expand the dimensions of the inputs to be used with Conv1D layers
X = dataset[:,0:len(dataset[0])-1]
X = expand_dims(X, axis=2)
y = dataset[:,len(dataset[0])-1]

print("X shape: ", X.shape)
print("y shape: ", y.shape)

#Establish the sample weights
thebins = np.linspace(0, 200, 100)
bkgPts = []
sigPts = []
for i in range(len(sampleData)):
    if y[i]==1:
        sigPts.append(sampleData[i][0])
    if y[i]==0:
        bkgPts.append(sampleData[i][0])
bkg_counts, binsbkg = np.histogram(bkgPts, bins=thebins)
sig_counts, binssig = np.histogram(sigPts, bins=thebins)
a = []
for i in range(len(bkg_counts)):
    tempSig = float(sig_counts[i])
    tempBkg = float(bkg_counts[i])
    if tempBkg!=0:
        a.append(tempSig/tempBkg)
    if tempBkg==0:
        a.append(0)
        
#Normalize the sample weights above a certain pT
for i in range(42,len(a)):
    a[i]=0.7
    
#Add in the sample weights, 1-to-1 correspondence with training data
#Sample weight of all signal events being equal to 1
#Sample weight of all background events being equal to the sig/bkg ratio at that jet's pT
weights = []
for i in range(len(sampleData)):
    if y[i]==1:
        weights.append(1)
    if y[i]==0:
        jetPt = sampleData[i][0]
        tempPt = int(jetPt/2)
        if tempPt>98:
            tempPt = 98
        weights.append(a[tempPt])

X shape:  (200000, 140, 1)
y shape:  (200000,)


In [3]:
#Define the model
model = Sequential()
model.add(Conv1D(filters=20, kernel_size=14, strides=14, activation='relu',input_shape=(len(dataset[0])-1,1)))
model.add(Conv1D(filters=10, kernel_size=1,activation='relu'))
model.add(Flatten())
model.add(Dense(20, activation = 'relu'))
model.add(Dense(20, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['binary_accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 10, 20)            300       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 10, 10)            210       
_________________________________________________________________
flatten (Flatten)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 20)                2020      
_________________________________________________________________
dense_1 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 2,971
Trainable params: 2,971
Non-trainable params: 0
______________________________________________________

In [4]:
#Train the network
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', verbose=1, patience=5)

model.fit(X,y,
          epochs=50,
          batch_size=50,
          verbose=2,
          sample_weight=np.asarray(weights),
          validation_split=0.20,
          callbacks=[callback])

Epoch 1/50
3200/3200 - 10s - loss: 0.4607 - binary_accuracy: 0.6241 - val_loss: 0.4536 - val_binary_accuracy: 0.6507
Epoch 2/50
3200/3200 - 6s - loss: 0.4499 - binary_accuracy: 0.6430 - val_loss: 0.4569 - val_binary_accuracy: 0.6697
Epoch 3/50
3200/3200 - 6s - loss: 0.4474 - binary_accuracy: 0.6433 - val_loss: 0.4527 - val_binary_accuracy: 0.6170
Epoch 4/50
3200/3200 - 6s - loss: 0.4456 - binary_accuracy: 0.6441 - val_loss: 0.4534 - val_binary_accuracy: 0.6660
Epoch 5/50
3200/3200 - 6s - loss: 0.4434 - binary_accuracy: 0.6463 - val_loss: 0.4489 - val_binary_accuracy: 0.6354
Epoch 6/50
3200/3200 - 6s - loss: 0.4417 - binary_accuracy: 0.6483 - val_loss: 0.4464 - val_binary_accuracy: 0.6358
Epoch 7/50
3200/3200 - 7s - loss: 0.4385 - binary_accuracy: 0.6515 - val_loss: 0.4448 - val_binary_accuracy: 0.6542
Epoch 8/50
3200/3200 - 7s - loss: 0.4356 - binary_accuracy: 0.6562 - val_loss: 0.4420 - val_binary_accuracy: 0.6399
Epoch 9/50
3200/3200 - 6s - loss: 0.4323 - binary_accuracy: 0.6628 - va

In [5]:
#model.save('../saved_model/L1BTagModel_Flatten_LargerTinyv3.h5')

## 2. Make performance plot for the model

In [9]:
#Establish the input and output data for each dataset, expanding dimensions for compatability with Conv1D layer input
def load_testing_data(data_path):
    
    with h5py.File(data_path, 'r') as hf:
        dataset = hf["Testing Data"][:]
    X_test = dataset[:,0:len(dataset[0])-1]
    X_test = expand_dims(X_test, axis=2)
    y_test = dataset[:,len(dataset[0])-1]
    
    return X_test, y_test

X_test, y_test = load_testing_data('../data/testingDataver3.h5')

In [ ]:
#Load in respective model for the datasets
model_1 = load_model('../saved_model/L1BTagModel_100GRU.h5')
model_2 = model

#Creating ROC curves based on model predictions for each dataset
y_pred_1 = model_1.predict(X_test).ravel()
y_pred_2 = model_2.predict(X_test).ravel()

fpr_1, tpr_1, thresholds_1 = roc_curve(y_test, y_pred_1)
fpr_2, tpr_2, thresholds_2 = roc_curve(y_test, y_pred_2)

auc_1 = auc(fpr_1, tpr_1)
auc_2 = auc(fpr_2, tpr_2)

In [ ]:
#Create plot for ROC
plt.figure(1)
plt.plot([0,1],[0,1],'k--')

plt.plot(fpr_1, tpr_1, label='Conv1D 100GRU, AUC={:.3f}'.format(auc_1))
plt.plot(fpr_2, tpr_2, label='Tiny Conv1D Flatten, AUC={:.3f}'.format(auc_2))

#Establish labels and save image
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.legend(loc='best')

#p_path = save_path + "v2LargerTinyConv1D_100GRU_Flatten.pdf"
#print(p_path)
#plt.savefig(p_path, bbox_inches='tight')